In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.12.0


In [2]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
               dropout, return_embeds=False):
    super(GCN, self).__init__()
    # We shall define the neural network architecture here that will be important for the forward pass
    self.convs = torch.nn.ModuleList([GCNConv(input_dim, hidden_dim)] +
                                     [GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers-2)] +
                                     [GCNConv(hidden_dim, output_dim)])
    self.bns = torch.nn.ModuleList([torch.nn.BatchNorm1d(hidden_dim) for _ in range(num_layers-1)])
    self.softmax = torch.nn.LogSoftmax(dim=1)
    self.dropout = dropout
    self.return_embeds = return_embeds

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()
    for bn in self.bns:
      bn.reset_parameters()
  
  def forward(self, x, adj_t):
    out = None
    for conv, bn in zip(self.convs[:-1], self.bns):
      x = conv(x, adj_t)
      x = bn(x)
      x = F.relu(x)
      x = F.dropout(x,p=self.dropout,training=self.training)
    x = self.convs[-1](x, adj_t)
    out = x if self.return_embeds else self.softmax(x)
    return out

In [3]:
def train(model, data, train_idx, optimizer, loss_fn):
  model.train()
  loss = 0
  
  optimizer.zero_grad()
  out = model(data.x, data.adj_t)
  loss = loss_fn(out[train_idx], data.y[train_idx].squeeze())
  
  loss.backward()
  optimizer.step()
  return loss.item()

In [4]:
# Test function here
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # TODO: Implement a function that tests the model by 
    # using the given split_idx and evaluator.
    model.eval()

    # The output of model on all data
    out = None

    ############# Your code here ############
    ## (~1 line of code)
    ## Note:
    ## 1. No index slicing here
    out = model(data.x, data.adj_t)
    #########################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # Save locally as csv
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)


    return train_acc, valid_acc, test_acc
  

In [5]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

if 'IS_GRADESCOPE_ENV' not in os.environ:
  dataset_name = 'ogbn-arxiv'
  dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())
  data = dataset[0]

  # Make the adjacency matrix to symmetric
  data.adj_t = data.adj_t.to_symmetric()

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # If you use GPU, the device should be cuda
  print('Device: {}'.format(device))

  data = data.to(device)
  split_idx = dataset.get_idx_split()
  train_idx = split_idx['train'].to(device)

Device: cuda


In [6]:
# Please do not change the args
if 'IS_GRADESCOPE_ENV' not in os.environ:
  args = {
      'device': device,
      'num_layers': 3,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 100,
  }
  args

In [7]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  model = GCN(data.num_features, args['hidden_dim'],
              dataset.num_classes, args['num_layers'],
              args['dropout']).to(device)
  evaluator = Evaluator(name='ogbn-arxiv')

In [8]:
import copy
if 'IS_GRADESCOPE_ENV' not in os.environ:
  # reset the parameters to initial random value
  model.reset_parameters()

  optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
  loss_fn = F.nll_loss

  best_model = None
  best_valid_acc = 0

  for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, data, train_idx, optimizer, loss_fn)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 4.2313, Train: 21.68%, Valid: 26.97% Test: 24.27%
Epoch: 02, Loss: 2.3564, Train: 24.07%, Valid: 22.97% Test: 28.06%
Epoch: 03, Loss: 1.9433, Train: 26.65%, Valid: 24.03% Test: 29.25%
Epoch: 04, Loss: 1.7527, Train: 34.99%, Valid: 30.27% Test: 34.57%
Epoch: 05, Loss: 1.6576, Train: 37.89%, Valid: 33.40% Test: 31.67%
Epoch: 06, Loss: 1.5891, Train: 41.71%, Valid: 35.46% Test: 33.42%
Epoch: 07, Loss: 1.5130, Train: 41.68%, Valid: 29.31% Test: 27.20%
Epoch: 08, Loss: 1.4587, Train: 40.68%, Valid: 26.02% Test: 23.04%
Epoch: 09, Loss: 1.4214, Train: 40.69%, Valid: 27.28% Test: 24.76%
Epoch: 10, Loss: 1.3853, Train: 41.22%, Valid: 33.24% Test: 35.89%
Epoch: 11, Loss: 1.3488, Train: 40.65%, Valid: 33.74% Test: 37.47%
Epoch: 12, Loss: 1.3183, Train: 40.35%, Valid: 34.05% Test: 38.26%
Epoch: 13, Loss: 1.2979, Train: 40.61%, Valid: 35.94% Test: 40.11%
Epoch: 14, Loss: 1.2761, Train: 40.62%, Valid: 36.83% Test: 41.19%
Epoch: 15, Loss: 1.2575, Train: 41.55%, Valid: 38.19% Test: 43